<a href="https://colab.research.google.com/github/hejasevis/mental-health-assistant/blob/main/mental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers peft bitsandbytes accelerate gradio

import pandas as pd
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Fo

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

data1 = pd.read_json('/content/drive/MyDrive/mentaldataset/intents.json')
data2 = pd.read_json('/content/drive/MyDrive/mentaldataset/Mental_Health_QA.json')
data3 = pd.read_csv('/content/drive/MyDrive/mentaldataset/train.csv')

data1 = data1.rename(columns={"intents": "instruction"})
data2 = data2.rename(columns={"intents": "instruction"})

data3 = data3.rename(columns={"Context": "context", "Response": "response"})

data1["context"] = ""
data1["response"] = ""

data2["context"] = ""
data2["response"] = ""


combined_data = pd.concat([data1, data2, data3], ignore_index=True)

print(combined_data.head())
from datasets import Dataset

dataset = Dataset.from_pandas(combined_data)

print(dataset)

def _add_text(rec):
    instruction = rec["instruction"]
    context = rec.get("context", "")
    response = rec.get("response", "")


    if context:
        rec["prompt"] = f"Instruction: {instruction}\nContext: {context}\nResponse:"
    else:
        rec["prompt"] = f"Instruction: {instruction}\nResponse:"

    rec["answer"] = response


    if rec["answer"] is None:
        rec["answer"] = ""

    rec["text"] = rec["prompt"] + rec["answer"]
    return rec

dataset = dataset.map(_add_text)
from huggingface_hub import login
login(token='')
from transformers import AutoTokenizer
import copy

Mounted at /content/drive
                                         instruction context response
0  {'tag': 'greetings', 'patterns': ['hello', 'he...                 
1  {'tag': 'goodbye', 'patterns': ['cya', 'see yo...                 
2  {'tag': 'self-esteem-A', 'patterns': ['How do ...                 
3  {'tag': 'self-esteem-B', 'patterns': ['How can...                 
4  {'tag': 'relationship-b', 'patterns': ['Why do...                 
Dataset({
    features: ['instruction', 'context', 'response'],
    num_rows: 3686
})


Map:   0%|          | 0/3686 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
import torch
import gradio as gr

# LoRA Konfigürasyonu
define_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# LoRA modeli
lora_model = get_peft_model(model, define_config)

# Kısa ve yönlendirici bir sistem promptu
system_prompt = (
    "You are a helpful assistant. Be concise, actionable, and motivational. "
    "Always end the response with an encouraging and motivational message such as 'You got this!'. "
    "Avoid asking questions or prompting further user interactions. Conclude firmly and positively."
)

def generate_response(age, gender, profession, issue):

    if not age.strip() or not gender.strip() or not profession.strip() or not issue.strip():
        return "Please make sure you have filled in all fields: Age, Gender, Profession, Issue."


    user_input = f"Age: {age}\nGender: {gender}\nProfession: {profession}\nIssue: {issue}"
    full_input = f"{system_prompt}\nUser:\n{user_input}\nAssistant:"


    inputs = tokenizer(
        full_input,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )

    with torch.no_grad():
        outputs = lora_model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=512,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
            temperature=0.7,
            top_p=0.85,
            top_k=30,
            repetition_penalty=1.1
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    response = response.split("Assistant:")[-1].strip()
    return response

In [ ]:
# Gradio arayüzü
demo = gr.Interface(
    fn=generate_response,
    inputs=[
        gr.Textbox(label="Age", placeholder="Example: 25,34 ..."),
        gr.Textbox(label="Gender", placeholder="Example: Male, Female ..."),
        gr.Textbox(label="Profession", placeholder="Example: Engineer, Student ..."),
        gr.Textbox(label="Issue", placeholder="Please specify the issue you are experiencing.")
    ],
    outputs=gr.Textbox(label="Response"),
    title="Mental Health Assistant",
    description=(
        "Hello! I'm here to assist you. Could you please tell me about the issue you're currently facing? "
        "To help me understand better, it would also be great if you could share your gender, age, and profession."
    )
)

# Uygulamayı başlatma
demo.launch(server_name="0.0.0.0")

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc35947940e28a4c63.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
